## Angelo Kelvakis
### Predicting Sexual Violence: Unconsenting Media  Google Scrape

This script was used to generate missing release year data and generate overview data of the media items.
NOTE: In order to run properly, javascript must be disabled on your browser in order to make html parsable.

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import re

In [ ]:
# Load data from Unconsenting Media found HERE: https://www.unconsentingmedia.org/list
UM_Data = pd.read_csv('Data.csv')
UM_Data = UM_Data.drop(['cleanName','cleanNameArticles','altName','comment','author','timeInFilm','posterUrl','youTubeUrl','createdAt','updatedAt'], axis=1)
# replace all 0s in the 'yearOfRelease' column with NaNs
UM_Data = UM_Data.replace(0,np.NaN)
# create subsetted df with missing years
Years_sv = UM_Data[UM_Data['yearOfRelease'].isna()]

### Scrape Google for Release Year
CAUTION: Estimated run time = 4.5 hrs

In [ ]:
df_movies = Years_sv

url_main = "https://www.google.com/search?q="

leftover_data = []
for i in df_movies['name']:
    # Append the url string with the name of the movie
    url_bucket = []
    # Remove any punctuation from the name of the media
    s = re.sub(r'[^\w\s]', '', i)
    for w in s.split():
        url_bucket.append(w)
        url_bucket.append("+")
    # get published date for books
    if df_movies[df_movies['name']==i]['itemType'].iloc[0] == 'book':
        url_bucket_m = url_main + ''.join(url_bucket)+'Published'
        flag = 'book'
    if df_movies[df_movies['name']==i]['itemType'].iloc[0] == 'movie':
        url_bucket_m = url_main + ''.join(url_bucket)+'movie+release+date'
        flag = 'movie'
    if df_movies[df_movies['name']==i]['itemType'].iloc[0] == 'TV show':
        url_bucket_m = url_main + ''.join(url_bucket)+'TV+Show+release+date'
        flag = 'TV show'
    # obtain html
    page = requests.get(url_bucket_m)
    soup = BeautifulSoup(page.content, "html.parser")
    # find the description from the page
    data = soup.findAll("div", class_="BNeawe iBp4i AP7Wnd")
    # troubleshoot misclassified movies
    if len(data) == 0:
        # misclassified tv show
        if flag == 'movie':
            url_bucket_m = url_main + ''.join(url_bucket)+'TV+Show+release+date'
            page = requests.get(url_bucket_m)
            soup = BeautifulSoup(page.content, "html.parser")
            data = soup.findAll("div", class_="BNeawe iBp4i AP7Wnd")
            # if release date not found, make NA
            if len(data) == 0:
                Year = None
                leftover_data.append([i,Year])
                print(i,"--",Year)
                continue
        # misclassified movie
        if flag == 'TV show':
            url_bucket_m = url_main + ''.join(url_bucket)+'movie+release+date'
            page = requests.get(url_bucket_m)
            soup = BeautifulSoup(page.content, "html.parser")
            data = soup.findAll("div", class_="BNeawe iBp4i AP7Wnd")
            # if release date not found, make NA
            if len(data) == 0:
                Year = None
                leftover_data.append([i,Year])
                print(i,"--",Year)
                continue
        # missclassified book (try TV show, then movie)
        if flag == 'book':
            url_bucket_m = url_main + ''.join(url_bucket)+'TV+Show+release+date'
            page = requests.get(url_bucket_m)
            soup = BeautifulSoup(page.content, "html.parser")
            data = soup.findAll("div", class_="BNeawe iBp4i AP7Wnd")
            if len(data) == 0:
                url_bucket_m = url_main + ''.join(url_bucket)+'movie+release+date'
                page = requests.get(url_bucket_m)
                soup = BeautifulSoup(page.content, "html.parser")
                data = soup.findAll("div", class_="BNeawe iBp4i AP7Wnd")
                # if release date not found, make NA
                if len(data) == 0:
                    Year = None
                    leftover_data.append([i,Year])
                    print(i,"--",Year)
                    continue
    Year = data[0].text

    leftover_data.append([i,Year])
    print(i,"--",Year)

df = pd.DataFrame(leftover_data,columns =['Title', 'Year'])

### Clean Data & Rebind

In [ ]:
# Remove NaN rows without a Year
df2=df.dropna(axis=0)
df2.shape
# Convert 'Year' column into just year
df2['Year'] = [re.search(r'[12]\d{3}', y).group(0) for y in df2['Year']]

# Rebind the Year data to redo_sv where the title matches
df2 = df2.rename(columns = {'Title':'name','Year':'yearOfRelease'})
# bind data and leave Nas
df3 = pd.merge(redo_sv, df2, on="name",how="outer")
# Fill those NAs with data from binded column
df3['yearOfRelease_x'] = df3['yearOfRelease_x'].fillna(df3['yearOfRelease_y'])
# Fix column names and drop extra column
df3 = df3.drop(['yearOfRelease_y'], axis=1)
df3 = df3.rename(columns = {'yearOfRelease_x':'yearOfRelease'})

# Drop rows with remainder of NAs in Year column
df3=df3.dropna(axis=0)

### Scrape Google for overview data
CAUTION: Estimated runtime 4 hrs 

In [ ]:
# find missing overviews 
missing_over = df3[df3['overview'].isna()]

In [ ]:
df_movies = missing_over

url_main = "https://www.google.com/search?q="

leftover_data = []
for i in df_movies['title']:
    # Append the url string with the name of the movie
    url_bucket = []
    # Remove any punctuation from the name of the media
    s = re.sub(r'[^\w\s]', '', i)
    for w in s.split():
        url_bucket.append(w)
        url_bucket.append("+")
    # get published date for books
    if df_movies[df_movies['title']==i]['itemType'].iloc[0] == 'book':
        url_bucket_m = url_main + ''.join(url_bucket)+str(int(df_movies[df_movies['title']==i]['yearOfRelease'].iloc[0]))+'+book+Synopsis'
        flag = 'book'
        # obtain html
        page = requests.get(url_bucket_m)
        soup = BeautifulSoup(page.content, "html.parser")
        # find the description from the page
        data = soup.findAll("div", class_="BNeawe s3v9rd AP7Wnd")
        
    if df_movies[df_movies['title']==i]['itemType'].iloc[0] == 'movie':
        url_bucket_m = url_main + ''.join(url_bucket)+str(int(df_movies[df_movies['title']==i]['yearOfRelease'].iloc[0]))+'+movie+overview'
        flag = 'movie'
        # obtain html
        page = requests.get(url_bucket_m)
        soup = BeautifulSoup(page.content, "html.parser")
        # find the description from the page
        data = soup.findAll("div", class_="BNeawe iBp4i AP7Wnd")
        
    if df_movies[df_movies['title']==i]['itemType'].iloc[0] == 'TV show':
        url_bucket_m = url_main + ''.join(url_bucket)+str(int(df_movies[df_movies['title']==i]['yearOfRelease'].iloc[0]))+'+TV+Show+Synopsis'
        flag = 'TV show'
        # obtain html
        page = requests.get(url_bucket_m)
        soup = BeautifulSoup(page.content, "html.parser")
        # find the description from the page
        data = soup.findAll("div", class_="BNeawe s3v9rd AP7Wnd")
    
    # troubleshoot misclassified media
    if len(data) == 0:
        # misclassified tv show
        if flag == 'movie':
            url_bucket_m = url_main + ''.join(url_bucket)+str(int(df_movies[df_movies['title']==i]['yearOfRelease'].iloc[0]))+'+TV+Show+summary'
            page = requests.get(url_bucket_m)
            soup = BeautifulSoup(page.content, "html.parser")
            data = soup.findAll("div", class_="BNeawe s3v9rd AP7Wnd")
            # if release date not found, make NA
            if len(data) == 0:
                Overview = None
                leftover_data.append([i,Overview])
                print(i,"--",Overview)
                continue
        # misclassified movie
        if flag == 'TV show':
            url_bucket_m = url_main + ''.join(url_bucket)+str(int(df_movies[df_movies['title']==i]['yearOfRelease'].iloc[0]))+'+movie+overview'
            page = requests.get(url_bucket_m)
            soup = BeautifulSoup(page.content, "html.parser")
            data = soup.findAll("div", class_="BNeawe iBp4i AP7Wnd")
            # if release date not found, make NA
            if len(data) == 0:
                Overview = None
                leftover_data.append([i,Overview])
                print(i,"--",Overview)
                continue
        # missclassified book (try TV show, then movie)
        if flag == 'book':
            url_bucket_m = url_main + ''.join(url_bucket)+str(int(df_movies[df_movies['title']==i]['yearOfRelease'].iloc[0]))+'+TV+Show+summary'
            page = requests.get(url_bucket_m)
            soup = BeautifulSoup(page.content, "html.parser")
            data = soup.findAll("div", class_="BNeawe s3v9rd AP7Wnd")
            if len(data) == 0:
                url_bucket_m = url_main + ''.join(url_bucket)+str(int(df_movies[df_movies['title']==i]['yearOfRelease'].iloc[0]))+'+movie+overview'
                page = requests.get(url_bucket_m)
                soup = BeautifulSoup(page.content, "html.parser")
                data = soup.findAll("div", class_="BNeawe s3v9rd AP7Wnd")
                # if release date not found, make NA
                if len(data) == 0:
                    Overview = None
                    leftover_data.append([i,Overview])
                    print(i,"--",Overview)
                    continue
    # Pull out first response                
    Overview = data[0].text
    
    # deal with IMBD data
    if '·' in Overview:
        imbdlist = data[0].text.split('\n')
        Overview = max(imbdlist,key=len)
        
    leftover_data.append([i,Overview])
    print(i,"--",Overview)
    #print(url_bucket_m)

df = pd.DataFrame(leftover_data,columns =['title', 'overview'])

### Clean Data and Rebind

In [ ]:
# bind data back to df3 to fill in NAs
final_df = pd.merge(df3, df, on="title",how="outer")
final_df['overview_x'] = final_df['overview_x'].fillna(final_df['overview_y'])
# Fix column names and drop extra column
final_df = final_df.drop(['overview_y'], axis=1)
final_df = final_df.rename(columns = {'overview_x':'overview'})

In [ ]:
final_df.to_csv('REDO_Overview_txtAnalysis')